In [6]:
import pandas as pd
import os
import numpy as np
from PyPDF2 import PdfReader
import re
import sys
from operator import itemgetter
import fitz
import json
import string

In [7]:
raw_data_path = "raw_data"

In [8]:
files = []
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for filename in filenames: 
        files.append(os.path.join(dirpath, filename))

In [9]:
pdfs = []
for file in files:
    if "pdf" in file:
        pdfs.append({"pdf": PdfReader(file),"path": file})

## OLD Approaches

In [10]:
pdf = pdfs[0]

In [11]:
pages = pdf['pdf'].pages

In [12]:
len(pages)

53

In [13]:
pages[0].extract_text()

' \n \n \nGuide on A rticle  1 \nof the  European  Convention  \non Human Rights  \nObligation to respect  human  rights  – \nConcepts of “jurisdiction ” \nand imputability  \nUpdated on 31 August  2023 \n \n \n \n \n \n \n \n \n \n \nPrepared by the Registry. It does not bind the Court . \n  '

In [14]:
sections = []
#for pdf in pdfs:
pdf = pdfs[0]
print("The case document path: ", pdf['path'])
print("The # of page in this document:", len(pdf['pdf'].pages))
for index, page in enumerate(pdf['pdf'].pages):
#page = pdf['pdf'].pages[5]
    text = page.extract_text()
    #splitted_text = re.findall('\n\d+\.  ', text)
    p = re.compile("\n\d+\.  ")
    for m in p.finditer(text):
        l = [int(s) for s in re.findall("\d+", m.group())]
        if l is not None:
            sections.append({'number': l[0], 'start': m.start(), 'page_num': index, 'text': text})

    #sections+=splitted_text


The case document path:  raw_data/Guide_Art_1_ENG.pdf
The # of page in this document: 53


In [ ]:
# #### ONLY FOR TEST PURPOSES ####
# sections = []
# page_num = 5
# page = pdf['pdf'].pages[page_num]
# text = page.extract_text()
# p = re.compile("\n\d+\.  ")
# start = 0
# for m in p.finditer(text):
#     end = m.start()
#     print(m.group())
#     l = [int(s) for s in re.findall("\d+", m.group())]
#     if l is not None:
#         sections.append({'number': l[0], 'start': m.start(), 'page_num': page_num})
# #splitted_text = re.findall('\n\d+\.  ', text)
# #sections+=splitted_text


In [15]:
print(sections)

[{'number': 1, 'start': 683, 'page_num': 2, 'text': 'Guide on Article 1 of the Convention – Obligation to respect human rights –  \nConcepts of “jurisdiction” and imputability  \nEuropean Court of Huma n Rights  3/53 Last update: 31.08 .202 3 Table of contents  \nNote to readers  ................................ ................................ ..............................  5 \nIntroduction  ................................ ................................ ................................ ... 6 \nI.   The territoriality principle and the exceptions thereto  ..............................  9 \nA.   The territoriality principle and its scope  ................................ ................................ .....................  9 \n1.   The territoria lity principle in the traditional sense of the term  ................................ ...........  9 \n2.   Transfers of sovereignty over a given territory  ................................ ................................ .. 12 \n3.   

In [16]:
df = pd.DataFrame(sections)

In [17]:
print(df.columns)

Index(['number', 'start', 'page_num', 'text'], dtype='object')


In [18]:
df = df[df['page_num'] >= 4 ]
df['text'][55]


'Guide on Article 1 of the Convention – Obligation to respect human rights –  \nConcepts of “jurisdiction” and imputability  \nEuropean Court of Huma n Rights  16/53 Last update: 31.08 .202 3 indeed under an obligation to provide legal assistance as the S tate in whose  territory the relevant \nevidence was to be found (§§ 243-245).  \n42.  In the case of  Aliyeva and Aliyev  v. Azerbaijan , 2014, the Court  had before it an application from \nthe parents of an  Azerbaijani  national who had been killed in Ukraine under  circumstances  \nimplicating  two other  Azerbaijani  national s. Pursuant  to a mutual legal assistance agreement \nbetween Ukraine and Azerbaijan , the case had been  transmitted  to Azerbaijan , but in the absence of \nevidence the  Azerbaijani  authorities  had discontinued the proceedings  against the  suspect s. The \nCourt  raised  of its own motion  the issue of its jurisdiction  ratione loci , considering  that in so far as  \nAzerbaijan  had accepted  the obl

In [19]:
#df['end'] = df['start'].shift(-1)
df['end'] = df.groupby('page_num')['start'].shift(-1)
df=df.dropna()
df

,number,start,page_num,text,end
13,1,532,5,Guide on Article 1 of the Convention – Obligat...,1199.0
14,2,1199,5,Guide on Article 1 of the Convention – Obligat...,2732.0
16,5,765,6,Guide on Article 1 of the Convention – Obligat...,1166.0
17,6,1166,6,Guide on Article 1 of the Convention – Obligat...,1392.0
18,7,1392,6,Guide on Article 1 of the Convention – Obligat...,2371.0
...,...,...,...,...,...
172,152,2483,46,Guide on Article 1 of the Convention – Obligat...,3096.0
173,153,3096,46,Guide on Article 1 of the Convention – Obligat...,4288.0
175,155,330,47,Guide on Article 1 of the Convention – Obligat...,992.0
176,156,992,47,Guide on Article 1 of the Convention – Obligat...,1845.0


In [20]:
real_data = []
real_index = 1
for index, row in df.iterrows():
    if real_index == 1 and real_index < row['number']:
        continue
    elif real_index > row['number']:
        continue
    else:
        text = row['text']
        real_data.append({ 'number': real_index, 'page_num': row['page_num'], 'text': text[int(row['start']):int(row['end'])], 'start': row['start'], 'end': row['end'] })
        real_index += 1 

In [21]:
data = pd.DataFrame(real_data)

In [22]:
data

,number,page_num,text,start,end
0,1,5,"\n1. As provided by Article 1, the engagement...",532,1199.0
1,2,5,"\n2. In the Convention context , the term ...",1199,2732.0
2,3,6,\n5. The travaux préparatoires go on to note...,765,1166.0
3,4,6,\n6. The concept of “jurisdiction” for the pu...,1166,1392.0
4,5,6,\n7. Establishing the existence of “jurisdict...,1392,2371.0
...,...,...,...,...,...
105,106,46,\n152. The Court takes into account reports a...,2483,3096.0
106,107,46,\n153. The Court has thus often attached impo...,3096,4288.0
107,108,47,\n155. The direct evidence of witnesses is al...,330,992.0
108,109,47,\n156. The Court may also rely on witness sta...,992,1845.0


In [23]:
data.to_csv("./data.csv")

### Citation centric regex

In [ ]:
key = '/Annots'
ank = '/A'
uri = '/URI'

In [ ]:
def get_hyperlinks_in_a_page(doc, page_number, key, uri, ank):
    urls = []
    
    # Access the specific page using list-like indexing
    page = doc.pages[page_number]
    
    # Extract the page's object dictionary
    pageObject = page.get_object()
    
    # Check if the key exists in the page's object dictionary
    if key in pageObject:
        ann = pageObject[key]
        
        # Iterate over the annotations
        for a in ann:
            u = a.get_object()
            
            # Check if the URI is in the annotation dictionary
            if uri in u and ank in u[uri]:
                urls.append(u[uri][ank])
    
    return urls

In [24]:
doc = pdfs[0]['pdf']
page = 9

In [25]:
urls = get_hyperlinks_in_a_page(doc, page, key, uri, ank)

NameError: name 'get_hyperlinks_in_a_page' is not defined

In [26]:
urls

NameError: name 'urls' is not defined

In [ ]:
urls[0].keys()

In [ ]:
urls[0]['/A']['/URI']

In [ ]:
urls[0]['/Rect']

In [28]:
doc = fitz.open("raw_data/Guide_Art_1_ENG.pdf")

In [29]:
len(doc)

53

In [30]:
page = doc[51]

In [31]:
page.get_links()

[{'kind': 2,
  'xref': 753,
  'from': Rect(69.75, 93.969970703125, 275.5199890136719, 107.39996337890625),
  'uri': 'http://hudoc.echr.coe.int/eng?i=001-70236',
  'id': ''},
 {'kind': 2,
  'xref': 754,
  'from': Rect(69.75, 107.39996337890625, 148.1300048828125, 120.8299560546875),
  'uri': 'http://hudoc.echr.coe.int/eng?i=001-69022',
  'id': ''},
 {'kind': 2,
  'xref': 755,
  'from': Rect(69.75, 172.79998779296875, 185.63999938964844, 186.22998046875),
  'uri': 'http://hudoc.echr.coe.int/eng?i=001-81672',
  'id': ''},
 {'kind': 2,
  'xref': 756,
  'from': Rect(69.75, 186.22998046875, 354.8399963378906, 199.65997314453125),
  'uri': 'http://hudoc.echr.coe.int/eng?i=001-183205',
  'id': ''},
 {'kind': 2,
  'xref': 757,
  'from': Rect(69.75, 251.6300048828125, 253.52999877929688, 265.05999755859375),
  'uri': 'http://hudoc.echr.coe.int/eng?i=001-103604',
  'id': ''},
 {'kind': 2,
  'xref': 763,
  'from': Rect(69.75, 317.02996826171875, 181.99000549316406, 330.4599914550781),
  'uri': 'ht

In [33]:
rect = page.get_links()[2]['from']
text = page.get_textbox(rect)
text

'Pad and Others v. Turkey '

In [34]:
pageSliced = doc.getPage(page)
pageObject = pageSliced.getObject()


AttributeError: 'Document' object has no attribute 'getPage'

In [ ]:
pageObject.getContents()

In [ ]:
if key in pageObject.keys():
    ann = pageObject[key]
    for a in ann:
        u = a.getObject()
        if uri in u[ank].keys():
            print(u['/StructParent'])

## Black Box Solutions

In [4]:

def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles


def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag


def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [5]:
doc = fitz.open(pdfs[0]['path'])
font_counts, styles = fonts(doc, granularity=False)

size_tag = font_tags(font_counts, styles)

elements = headers_para(doc, size_tag)

with open("doc.json", 'w') as json_out:
    json.dump(elements, json_out)

NameError: name 'pdfs' is not defined

## Current Approach

In [ ]:
from tika import parser
import unidecode

In [ ]:
pdf_sample_number = 26

In [ ]:
parsedPDF   = parser.from_file(pdfs[pdf_sample_number]['path'])
content     = parsedPDF['content']
metadata    = parsedPDF['metadata']
print(pdfs[pdf_sample_number]['path'])

In [ ]:
#content

## Finding Hierarchy

#### Operations and Helper Functions

In [ ]:
def int_to_roman(num):
 
    # Storing roman values of digits from 0-9
    # when placed at different places
    m = ["", "M", "MM", "MMM"]
    c = ["", "C", "CC", "CCC", "CD", "D",
         "DC", "DCC", "DCCC", "CM "]
    x = ["", "X", "XX", "XXX", "XL", "L",
         "LX", "LXX", "LXXX", "XC"]
    i = ["", "I", "II", "III", "IV", "V",
         "VI", "VII", "VIII", "IX"]
 
    # Converting to roman
    thousands = m[num // 1000]
    hundreds = c[(num % 1000) // 100]
    tens = x[(num % 100) // 10]
    ones = i[num % 10]
 
    ans = (thousands + hundreds +
           tens + ones)
 
    return ans

In [ ]:
roman_num_upper_list = {}
roman_num_lower_list = {}
for num in range(1,20):
    roman_num_upper = int_to_roman(num)+"."
    roman_num_lower = str.lower(roman_num_upper) 
    roman_num_upper_list[roman_num_upper] = num
    roman_num_lower_list[roman_num_lower] = num

#### Header Hierarchy Extraction

In [ ]:
#table_of_content_pattern = '.+\.   [^.]+[\.]{5,1000} \d+'
#table_of_content_start_pattern = '.+\.   |\n{1,2} [A-Z]'
#table_of_content_end_pattern = '[\.]{5,1000}'
#list_of_cited_cases_pattern =  'List[^.]+[. ]*\d+'

#table_of_content_start_pattern = '.{1,4}\.   '
#table_of_content_start_pattern = '\n[IV]{1,3}\. {1,3}|\n[a-z]{1,3}\. {1,3}|\n[A-Z]{1,3}\. {1,3}|\n[0-9]{1,3}\. {1,3}'
table_of_content_end_pattern = '[.]{3,1000} *\d+| [.]{1,1000} *\d+'

#table_of_content_end_pattern = '[.]{0,1000} \d+ \n'
table_of_content_start_pattern = '\n.{1,3}\. {1,3}'
#table_of_content_end_pattern = '[.]{1,1000}'
list_of_cited_cases_pattern =  'List[^.]+[. ]*\d+'

#table_of_content = re.findall(table_of_content_end_pattern, content)

In [ ]:
start_pattern = re.compile(table_of_content_start_pattern)
end_pattern = re.compile(table_of_content_end_pattern)

In [ ]:
start_list = re.compile(list_of_cited_cases_pattern).search(content).span()[0]
start_of_the_guide = 0
for e in end_pattern.finditer(content):
    if e.start() > start_list:
        text = content[start_list:e.start()]
        text = text.replace("\n", "")
        list_of_cases_text = text
        start_of_the_guide = e.end()
        break

In [ ]:
content[start_list:re.compile(list_of_cited_cases_pattern).search(content).span()[1]]

In [ ]:
table_of_content = []
last_confirmed_end = 0
for s in start_pattern.finditer(content[:start_of_the_guide]):
    #print(content[s.start():s.end()])
    for e in  end_pattern.finditer(content):
        if e.start() > s.start():
            text = content[s.start():e.start()]
            text = text.replace("\n", "")
            table_of_content.append(text)
            last_confirmed_end = e.end()-1
            break

In [ ]:
table_of_content.append(list_of_cases_text)

In [ ]:
#content[:start_of_the_guide]

In [ ]:
(table_of_content)

In [ ]:
#content[:e.end()]

In [ ]:
clear_content = content[start_of_the_guide:]

In [ ]:
subheader_pattern = '[A-Z]\. *.+'
section_pattern = '[0-9]+\. *.+'
subsection_pattern = '[a-z]+\. *.+'
#subheader_pattern = '[A-Z]\. *.+'
#section_pattern = '[0-9]+\. *.+'
#subsection_pattern = '[a-z]+\. *.+'

In [ ]:
re.findall(subheader_pattern, 'A.   The concept of “finding of guilt” ')

In [ ]:
# a different subheader pattern
#subheader_pattern = '.+[\.]{1,1000}'

In [ ]:
special_case = False

In [ ]:
header = []
header_index = -1
subheader_index = -1
section_index = -1
for element in (table_of_content[:-1]):
    print(repr(element), header_index, subheader_index, section_index)
    element = element.rstrip(" ")
    # finding whether it's a header
    for roman_upper in roman_num_upper_list.keys():
        if roman_upper in element and roman_num_upper_list[roman_upper] > (header_index): # this element is a header
            header.append({"text": element.rstrip(".").rstrip(" "), "subheader": []})
            #print(element.rstrip(".").rstrip(" "))
            header_index += 1
            subheader_index = -1
            break
    else:
        # finding whether it's a subheader or section or subsection
        subheader_matcher = re.compile(subheader_pattern)
        section_matcher = re.compile(section_pattern)
        subsection_matcher = re.compile(subsection_pattern)
        if subheader_matcher.match(element): # this element is a subheader
            header[header_index]['subheader'].append({"text": element.rstrip(".").rstrip(" "), "section": []})
            subheader_index += 1
            section_index = -1
        elif section_matcher.match(element): # this element is a section
            if special_case and subheader_index == -1:
                header[header_index]['subheader'].append({"text": "", "section": []})
                subheader_index += 1
                section_index = -1
            header[header_index]['subheader'][subheader_index]['section'].append({"text": element.rstrip(".").rstrip(" "), "subsection": []})
            section_index += 1
        else: # this element is a subsection
            if special_case and section_index == -1:
                header[header_index]['subheader'][subheader_index]['section'].append({"text": "", "subsection": []})
                section_index += 1
            header[header_index]['subheader'][subheader_index]['section'][section_index]['subsection'].append({"text": element.rstrip(".").rstrip(" ")})
    
# special case list of cases header
header.append({"text": table_of_content[-1].rstrip(".").rstrip(" "), "subheader": []})

In [ ]:
header

### Parsing paragraphs

#### Getting content after the table of content section. Now we have hierarchy for the whole text

In [ ]:
http_pattern = r'http[s]?:\/\/hudoc.echr.coe.int\/[a-z]+\?i\=\d+\-\d+'
website_pattern = r'([^ :]*):\/\/(?:([^:]*):([^@]*)@|)([^/:]{1,}):?(\d*)?(\/[^? ]*)\??((?:[^=&# ]*=?[^&# ]*&?)*)#?([^ ]*)?'
footer_pattern = r" *Guide on Article \d+ of the Convention \- [A-Za-z ]+ European Court of Human Rights \d+\/\d+ Last update: \d+.\d+.\d+ *"
citation_pattern = "[A-Z]+[A-Za-z.\- ]*v\.[A-Za-z.\- ]+[GCIdecmeritsrevision\[\]().\d\- ]*[S,\d ]*[\-\d ]*[and]{0,3}[\-\d ]*"

In [ ]:
def text_cleaner(text, website_pattern, footer_pattern):
    sentences = re.sub("\n", " \n ", text)
    sentences = unidecode.unidecode(sentences)
    sentences = re.sub(website_pattern, '', sentences)
    sentences = re.sub(" \n ", "", sentences)
    sentences = re.sub(footer_pattern, ' ', sentences)
    return sentences

In [ ]:
def get_the_pattern(text):
    #print(text)
    #text = re.sub("-", " - ", text)
    lst = re.split(" ", text)
    r = [x for x in lst if len(x) > 0]
    pat = ""
    for i, e in enumerate(r):
        current_pat = e
        shifter = 0
        for p in re.compile("[()\[\]]").finditer(e):
            index = p.start() + shifter
            current_pat = current_pat[:index] + '\\' +current_pat[index:]
            #print(repr(current_pat))
            shifter +=1 
        if i < len(r) - 1:
            pat += current_pat + " *\n*"
            #pat += current_pat + "[\d{0,2}\n ]*"
        else:
            pat += current_pat
    #print(repr(pat))
    return pat

In [ ]:
for e in re.compile(get_the_pattern(header[2]["text"])).finditer(clear_content):
    print(e.group(), e.start(), e.end())

In [ ]:
#tmp = clear_content[41308:44158]
#tmp
#tmp_pat = get_the_pattern('4. Mental illness/measure of protection')
#tmp = "licences). \n\n4.  Proportionality and related issues (fair balance, compensation, margin of \nappreciation) \n\n142.  In order to be compatible with the general rule set forth in the first sentence of the first \nparagraph of Article 1 of Protocol No. 1, an interference with the right to the peaceful enjoyment of \n“possessions”, apart from being prescribed by law and in the public interest, must strike a “fair \nbalance” between the demands of the general interest of the community and the r"
#print(re.findall(tmp_pat,tmp))
#tmp_pat

In [ ]:
#re.findall(get_the_pattern(tmp_pat), tmp)

In [ ]:
titles = []
for head in header:
    current_pattern = get_the_pattern(head['text'])
    start = re.compile(current_pattern).search(clear_content).span()[0]
    end = re.compile(current_pattern).search(clear_content).span()[1]
    #print(re.findall(current_pattern, clear_content), "pattern:", repr(current_pattern))
    titles.append({
        "type": "header", 
        "start": start, 
        "end": end,
        "header": clear_content[start:end],
        "subheader": None, 
        "section": None,
        'subsection': None
        })
    for subheader in head['subheader']:
        current_pattern = get_the_pattern(subheader['text'])
        #print("\t", re.findall(current_pattern, clear_content), "pattern:",  repr(current_pattern))
        start = re.compile(current_pattern).search(clear_content).span()[0]
        end = re.compile(current_pattern).search(clear_content).span()[1]
        titles.append({
            "type": "subheader", 
            "start": start,
            "end": end, 
            "header": head['text'], 
            "subheader": clear_content[start:end],
            "section": None,
            'subsection': None
            })
        for section in subheader['section']:
            current_pattern = get_the_pattern(section['text'])
            #print("\t\t", re.findall(current_pattern, clear_content), "pattern:", repr(current_pattern))
            start = re.compile(current_pattern).search(clear_content).span()[0]
            end = re.compile(current_pattern).search(clear_content).span()[1]
            titles.append({
                "type": "section", 
                "start": start, 
                "end": end, 
                "header": head['text'], 
                "subheader": subheader['text'],
                "section": clear_content[start:end],
                "subsection": None
                })
            for subsection in section['subsection']:
                current_pattern = get_the_pattern(subsection['text'])
                #print("\t\t\t", re.findall(current_pattern, clear_content), "pattern:", repr(current_pattern))
                start = re.compile(current_pattern).search(clear_content).span()[0]
                end = re.compile(current_pattern).search(clear_content).span()[1]
                #print(re.compile(current_pattern).findall(content), start, end)
                titles.append({
                    "type": "subsection", 
                    "start": start, 
                    "end": end, 
                    "header": head['text'], 
                    "subheader": subheader['text'], 
                    "section": section['text'],
                    "subsection": clear_content[start:end], 
                    })


In [ ]:
titles

#### Hierarchy now is located in the text itself, now we will get the inner sections

##### Helper functions and Regexes

In [ ]:
footer_arr = ["Guide on Article 1 of the Convention – Obligation to respect human rights – Concepts of “jurisdiction” and imputability",
"Guide on Article 1 of Protocol No. 1 – Protection of property",
"Guide on Article 1 of Protocol No. 7 to the Convention – Procedural safeguards on expulsion"
"Guide on Article 2 of the Convention – Right to life",
"Guide on Article 2 of Protocol No. 1 – Right to education",
"Guide on Article 3 of the Convention – Prohibition of torture",
"Guide on Article 2 of Protocol No. 4 to the Convention – Freedom of movement",
"Guide on Article 3 of Protocol No. 1 – Right to free elections",
"Guide on Article 3 of Protocol N° 4 to the Convention – Prohibition of expulsion of nationals"
,"Guide on Article 4 of the Convention – Prohibition of slavery and forced labour"
,"Guide on Article 4 of Protocol No. 4 – Prohibition of collective expulsion of aliens"
,"Guide on Article 4 of Protocol No. 7 – Right not to be tried or punished twice"
,"Guide on Article 5 of the Convention – Right to liberty and security"
,"Guide on Article 6 of the Convention – Right to a fair trial (criminal limb)"
,"Guide on Article 6 of the Convention – Right to a fair trial (civil limb)"
,"Guide on Article 7 of the Convention – No punishment without law"
,"Guide on Article 8 of the Convention – Right to respect for private and family life"
,"Guide on Article 9 of the Convention – Freedom of thought, conscience and religion"
,"Guide to Article 10 of the Convention – Freedom of expression"
,"Guide on Article 11 of the Convention – Freedom of assembly and association"
,"Guide on Article 12 of the Convention – Right to marry"
,"Guide on Article 13 of the Convention – Right to an effective remedy"
,"Guide on Article 14 of the Convention (prohibition of discrimination) and on Article 1 of Protocol No. 12 (general prohibition of discrimination)"
,"Guide on Article 15 of the Convention – Derogation in time of emergency"
,"Guide on Article 17 of the Convention – Prohibition of abuse of rights"
,"Guide on Article 18 of the Convention – Limitation on use of restrictions on rights"
,"Guide on Article 46 of the Convention – Binding force and execution of judgments"]

In [ ]:
http_pattern = r'http[s]?:\/\/hudoc.echr.coe.int\/[a-z]+\?i\=\d+\-\d+'
website_pattern = r'([^ :]*):\/\/(?:([^:]*):([^@]*)@|)([^/:]{1,}):?(\d*)?(\/[^? ]*)\??((?:[^=&# ]*=?[^&# ]*&?)*)#?([^ ]*)?'
footer_pattern = r" *Guide [ton]{2} Article \d+ of [A-Za-z\(\)\d\.\",\- ]*European *Court *of *Human *Rights *\d+\/\d+ *Last *update: *\d+.\d+.\d+ *"
citation_pattern = "[A-Z]+[A-Za-z.\- ]*v\.[A-Za-z.\- ]+[GCIdecmeritsrevision\[\]().\d\- ]*[S,\d ]*[\-\d ]*[and]{0,3}[\-\d ]*"

In [ ]:
re.compile(footer_pattern).findall( 'Guide on Article 1 of Protocol No. 1 - Protection of property European Court of Human Rights 89/98 Last update: 31.08.2022 ')

In [ ]:
#generic = "[A-Za-z.\- ]*v\.[A-Za-z.\- ]+"
#gc = "[\[GC\]]*"
#dec = "[\(dec\.\)]*"
#year = "[ \d\d\d\d]*"
#par = "[ S{2,4}]*"
#revision = "[\(revision\)]*"
#no = "[\(no\. \d*\)]*"
#par_interval = "[ \d+\-*\d*]*"
#alternative_par_interval = "[and \d+\-*\d*]*"
#suffix_pattern = "where.*|which.*|which.*|in which.*"
#semi_column = "[;),]?"
#citation_pattern = generic + gc + dec + revision + no + ",*" + year + par + par_interval + alternative_par_interval + semi_column
#cleaner = "[ \.,:;()]*"
citation_pattern = "[A-Z]+[A-Za-z.\- ]*v\.[A-Za-z.\- ]+[GCIdecmeritsrevision\[\]().\d\- ]*[S,\d ]*[\-\d ]*[and]{0,3}[\-\d ]*"

In [ ]:
#titles

In [ ]:
#content[5666:6452]
range(len(titles) - 1)

In [ ]:
def suffix_checker(text):
    suffix_pattern_list = ("where", "which", "in which")
    return text.lstrip("[ \.,:;()]*").startswith(suffix_pattern_list)


In [ ]:
content = clear_content

##### Creating Dataset

In [ ]:
rule_num = 1
result_dic = []
first = False
#citation_str  = []
for index in range(len(titles) - 1):

    ## Getting the raw text between the titles
    if index < len(titles) - 1:
        #print(content[titles[index]['end']:titles[index+1]['start']])
        raw_text = content[titles[index]['end']:titles[index+1]['start']]
    #    raw_text = content[titles[index]['end']:]
        
    article_subs = re.split("\n\d+\. *", raw_text)[1:]
    nums = re.findall("\n\d+\. *", raw_text)

    for num, art in zip(nums, article_subs):
        if first:
            rule_num = int(re.findall("\d+", num)[0])
            first = False
        #print(num, ":", art)
            #print(rule_num, ':', art)
        if rule_num == int(re.findall("\d+", num)[0]) or (False and int(re.findall("\d+", num)[0]) > rule_num):
            rule_num = int(re.findall("\d+", num)[0])
            sentences = re.sub("\n", " \n ", art)
            sentences = unidecode.unidecode(sentences)
            sentences = re.sub(website_pattern, '', sentences)
            sentences = re.sub(" \n ", "", sentences)
            sentences = re.sub(footer_pattern, ' ', sentences)
            citations = re.findall(citation_pattern, sentences) 
            rules = re.split(citation_pattern, sentences)
            rule_index = 0
            for citation_index, citation in enumerate(citations):
                citation_type = "single"
                suffix = None
                if len(rules[citation_index]) > 5 and not suffix_checker(rules[citation_index]): # this is not a suffix as well 
                    rule = rules[citation_index]
                    #citation_str.append({"rule_num": rule_num,"text":rules[citation_index][-1]})
                    if len(rules) > citation_index + 1 and suffix_checker(rules[citation_index+1]): # there is a suffix after that sentence
                        suffix = rules[citation_index+1]
                    rule_index = citation_index
                else:
                    # multicitation or in sentence citation
                    if len(rules[citation_index].strip()) == 0 and citation_index == 0: # in sentence citation, just direct to the next sentence
                        rule_index += 1
                        rule = rules[rule_index]
                        citation_type = "in_sentence"
                    else:
                        citation_type = "multi"
                        result_dic[-1]['citation_type'] = citation_type
                        #if len(rules[citation_index+1]) >  0:
                        #    citation_str.append({"rule_num": rule_num,"text":rules[citation_index+1][:max(1, len(rules[citation_index+1]))]})
                        if len(rules) > citation_index + 1 and suffix_checker(rules[citation_index+1]):
                            suffix = rules[citation_index+1]
                        rule = rules[rule_index] # multi citation, just keep the old sentence

                result_dic.append({
                        "rule_num": rule_num, 
                        "sentence": rule, 
                        "citation": citation,
                        "citation_type": citation_type,
                        "suffix": suffix,
                        "header": titles[index]['header'], 
                        "subheader": titles[index]['subheader'],
                        'section': titles[index]['section'],
                        "subsection": titles[index]['subsection']
                        })
                #print(titles[index])
                
            rule_num += 1
        else:
            #print("not a rule")
            pass

In [ ]:
data = pd.DataFrame(result_dic)

In [ ]:
data.shape

In [ ]:
del data

In [ ]:
data[data['citation_type'] == "in_sentence"].shape

In [ ]:
data.to_csv("../example_output_new_regex.csv")

## Finding corresponding url

In [ ]:
url_citations_letter = []
p = re.compile("—[A-Z]—")
index = 0
for m in p.finditer(content):
    url_citations_letter.append({ 'url_citation_letter': m.group(), 'start': m.start(), 'end': 0})
    if index > 0:
        url_citations_letter[index-1]['end'] = m.start()
    index += 1    
url_citations_letter[index-1]['end'] = len(content)
#sections+=splitted_text

In [ ]:
footer_pattern_url = r" *Guide [ton]{2} Article \d+ of [A-Za-z\(\)\d\.\",\- ]*\n*European *\n*Court *\n*of *\n*Human *\n*Rights *\n*\d+\/\d+ *\n*Last *\n*update: *\n*\d+.\d+.\d+ *"

In [ ]:
re.compile("-[A-Z]-").search(unidecode.unidecode(content)).span()[0]

In [ ]:
tmp = re.findall(footer_pattern_url, unidecode.unidecode(content[436622:]))[0]
nums = re.findall('\d+\/\d+', tmp)[0]
nums

In [ ]:
doc = fitz.open(pdfs[1]['path'])

In [ ]:
page = doc[int(re.split("/", nums)[0]) - 2]

In [ ]:
page

In [ ]:
links = page.get_links()
links[0]

In [ ]:
for index in range(int(re.split("/", nums)[0]) - 2, len(doc)):
    page  = doc[index]
    links = page.get_links()
    for link in links:
        rect = link['from']
        text = page.get_textbox(rect)
        print(link['uri'], text)

In [ ]:
url_citations_text = [content[url_citation['start']:url_citation['end']] for url_citation in url_citations_letter]

In [ ]:
print(repr(url_citations_text[-2:]))

In [ ]:
len(url_citations_text)

In [ ]:
citation_letter_text = []
for every_letter in url_citations_text:
    footer_pattern_url = r" *Guide [ton]{2} Article \d+ of [A-Za-z\(\)\d\.\",\- ]*\n*European *\n*Court *\n*of *\n*Human *\n*Rights *\n*\d+\/\d+ *\n*Last *\n*update: *\n*\d+.\d+.\d+ *"
    current_letter = every_letter[1]
    sample_letter_text = unidecode.unidecode(every_letter)
    sample_letter_text = re.sub(footer_pattern_url,"", sample_letter_text)
    #print(re.findall(footer_pattern_url, sample_letter_text))
    interval_of_citations = []
    for letter in re.compile("\n"+current_letter).finditer(sample_letter_text):
        interval_of_citations.append({"start": letter.start(),"end":0})
    for index, interval in enumerate(interval_of_citations):
        if index < len(interval_of_citations)-1:
            interval_of_citations[index]['end'] = interval_of_citations[index+1]['start']
        else:
            interval_of_citations[index]['end'] = len(sample_letter_text) # we have to deal with that part because there could be many unnnecessary strings here!
        #print(repr(sample_letter_text[interval_of_citations[index]['start']: interval_of_citations[index]['end']]))
        current_citation = sample_letter_text[interval_of_citations[index]['start']: interval_of_citations[index]['end']]
        parts = re.split("[ \n]*v\.[ \n]*", current_citation)
        if len(parts) >= 2:
            clear_citation = parts[0]+" v. "+re.match("[^,]+",parts[1]).group()
        else:
            clear_citation = current_citation
        clear_citation = re.sub("\n", "", clear_citation)
        citation_letter_text.append(clear_citation)

citation_url = []
for every_letter in url_citations_text:
    sample_letter = re.split("\n", every_letter)
    for sample in sample_letter:
        sample = unidecode.unidecode(sample)
        if "http" in sample:
            citation_url.append(sample)
    
    
    #for sample in re.compile("\n"+current_letter+"^(?!.*(v\.))v\.[^,]+").finditer(sample_letter):


In [ ]:
len(citation_letter_text)

In [ ]:
len(citation_url)

In [ ]:
from collections import Counter
[k for k,v in Counter(citation_url).items() if v>1]

In [ ]:
#citation_url = []
#citation_letter_text = []
#for every_letter in url_citations_text:
#    sample_letter = re.split("\n", every_letter)
#    for sample in sample_letter:
#        sample = unidecode.unidecode(sample)
#        if "v." in sample:
#            citation_letter_text.append(sample)
#        elif "http" in sample:
#            citation_url.append(sample)


In [ ]:
citation_mapper = {}
for index, citation_text in  reversed(list(enumerate(citation_letter_text))):
    current_index = index+max(0, len(citation_url) - len(citation_letter_text))
    citation_mapper[citation_text] = citation_url[current_index]
citation_mapper

## Placing URL with mapper

In [ ]:
import jellyfish

In [ ]:
data['citation'][0]

In [ ]:
data['citation'][4]

In [ ]:
def citation_url_mapper(text, citation_mapper):
    max = 0
    current_best_distance_key = ""
    final_key = ""
    for key in citation_mapper.keys():
        current_distance = jellyfish.jaro_similarity(key, text)
        if max < current_distance:
            max = current_distance
            current_best_distance_key = key
        #    final_key = citation_mapper[key]
        if key in text:
            final_key = citation_mapper[key]
    else:
        final_key = citation_mapper[current_best_distance_key]
    return final_key

In [ ]:
data['citation_url'] = data.apply(lambda row: citation_url_mapper(row['citation'], citation_mapper=citation_mapper), axis=1)

In [ ]:
data.to_csv("../citation_oriented_raw_data_new_regex.csv")

## Data Cleaning and Manipulation 

### Clearing the Sentences, getting the rule type and getting the paragraph numbers from citations

In [ ]:
data.head()

In [ ]:
stripped_data = data.copy()

In [ ]:
stripped_data['sentence'] = data.apply(lambda row: row['sentence'].strip("[ \.,:;()]*"), axis=1)

In [ ]:
#stripped_data.apply(lambda row: "example" if "#" in row['sentence'] else "rule", axis=1)

In [ ]:
stripped_data['rule_type'] = stripped_data.apply(lambda row: "example" if "#" in row['sentence'] else "rule", axis=1)

In [ ]:
#stripped_data.apply(lambda row: print(row['citation'],":",re.compile(citation_number_pattern).findall(row['citation'])), axis=1)

In [ ]:
#citation_par_nums = []
#for index, row in stripped_data.iterrows():
#    citation_par_nums.append({"citation": row['citation'], "numbers": re.compile(citation_number_pattern).findall(row['citation'])})

In [ ]:
#citation_par_nums_df = pd.DataFrame(citation_par_nums)

In [ ]:
def capture_par_nums(row_text):
    citation_number_pattern = "S{2,4}.*"
    single = "\d{1,4}"
    double = "\d{1,4}\-\d{1,4}"
    nums = []
    for par_num_text in re.compile(citation_number_pattern).finditer(row_text):
        num_text = par_num_text.group()
        if "and" in num_text:
            for element in re.split(" *and *", num_text):
                if len(element) > 0:
                    interval = re.findall(single, element)
                    for num in range(int(interval[0]), int(interval[-1]) + 1):
                        nums.append(num)
        else:
            interval = re.findall(single, num_text)
            for num in range(int(interval[0]), int(interval[-1]) + 1):
                nums.append(num)
    return nums

In [ ]:
stripped_data['citation_par_nums'] = stripped_data.apply(lambda row: capture_par_nums(row['citation']), axis=1)

In [ ]:
#tmp = stripped_data[['rule_num','citation', 'citation_par_nums']]

In [ ]:
stripped_data.to_csv("../stripped_data_new_regex.csv")

## Data analysis

In [ ]:
stripped_data.columns

In [ ]:
stripped_data['rule_type'].value_counts()

In [ ]:
stripped_data['citation_type'].value_counts()

In [ ]:
stripped_data[stripped_data['citation_url'] == ""].shape

In [ ]:
stripped_data[stripped_data['citation_par_nums'].str.len() <= 0].shape

## Experiments with content string

#### Individual Paragraphs tests

In [ ]:
test_text = content[titles[6]['end']:titles[7]['start']]
#test_text = "\n\n205.  Thus, it has held that, in cases where negligence by a private individual resulted in the victim’s \ndeath or serious injury, the States’ duty to take appropriate steps to safeguard the lives of those \nwithin its jurisdiction includes an obligation to have in place an effective independent judicial system \nsecuring the availability of legal means capable of establishing the facts, holding accountable those \nat fault and providing appropriate redress to the victim (Fergec v. Croatia, § 32, which concerned the \neffectiveness of proceedings concerning the explosion of a grenade in a pizza parlour; Ciechońska \n\nhttp://hudoc.echr.coe.int/eng?i=001-189781\nhttp://hudoc.echr.coe.int/eng?i=001-189781\nhttp://hudoc.echr.coe.int/eng?i=001-208279\nhttp://hudoc.echr.coe.int/eng?i=001-207757\nhttp://hudoc.echr.coe.int/eng?i=001-207757\nhttp://hudoc.echr.coe.int/eng?i=001-207757\nhttp://hudoc.echr.coe.int/eng?i=001-208279\nhttp://hudoc.echr.coe.int/eng?i=001-120961\nhttp://hudoc.echr.coe.int/eng?i=001-173467\nhttp://hudoc.echr.coe.int/eng?i=001-105102\n\n\nGuide on Article 2 of the Convention – Right to life \n\nEuropean Court of Human Rights 44/56 Last update: 31.08.2022 \n\nv. Poland, 2011, § 66, which concerned proceedings regarding the death of the applicant’s husband \nafter being hit by a tree in a health resort; Anna Todorova v. Bulgaria, 2011, § 72, which concerned \nproceedings regarding the death of the applicant’s son in a road traffic accident; Ilbeyi Kemaloğlu \nand Meriye Kemaloğlu v. Turkey, § 38, which concerned the death of the applicant’s seven-year old \nson who froze to death while trying to walk home in a blizzard; Kotelnikov v. Russia, §§ 99-101, \nwhere the applicant was seriously injured in a traffic accident)."
test_text

In [ ]:
generic = "[A-Za-z.\- ]*v\.[A-Za-z\[\]\(\)\-\d\.S{2,4} ]+"
gc = "[\[GC\]]*"
dec = "[\(dec\.\)]*"
year = "[ \d\d\d\d]*"
par = "[ S{2,4}]*"
revision = "[\(revision\)]*"
no = "[\(no\. \d*\)]*"
par_interval = "[ \d+\-*\d*]*"
alternative_par_interval = "[and \d+\-*\d*]*"
suffix_pattern = "where.*|which.*|which.*|in which.*"
semi_column = "[;),]?"
citation_pattern = generic + gc + dec + revision + no + ",*" + year + par + par_interval + alternative_par_interval + semi_column
cleaner = "[ \.,:;()]*"

In [ ]:
citation_pattern = "[A-Z]+[A-Za-z.\- ]*v\.[A-Za-z.\- ]+[GCIdecmeritsrevision\[\]().\d\- ]*[S,\d ]*[\-\d ]*[and]{0,3}[\-\d ]*"

In [ ]:
suffix_pattern = "where.*|which.*|in which.*"
suffix_pattern_list = ("where", "which", "in which")

In [ ]:
http_pattern = r'http[s]?:\/\/hudoc.echr.coe.int\/[a-z]+\?i\=\d+\-\d+'
website_pattern = r'([^ :]*):\/\/(?:([^:]*):([^@]*)@|)([^/:]{1,}):?(\d*)?(\/[^? ]*)\??((?:[^=&# ]*=?[^&# ]*&?)*)#?([^ ]*)?'
footer_pattern = r" *Guide on Article \d+ of the Convention \- [A-Za-z ]+ European Court of Human Rights \d+\/\d+ Last update: \d+.\d+.\d+ *"

In [ ]:
article_subs = re.split("\n\d+\. ", test_text)[1:]
nums = re.findall("\n\d+\. ", test_text)

for num, art in zip(nums, article_subs):
    #print(num, ":", art)
    if True:#rule_num == int(re.findall("\d+", num)[0]):
        sentences = re.sub("\n", " \n ", art)
        sentences = unidecode.unidecode(sentences)
        sentences = re.sub(website_pattern, '', sentences)
        sentences = re.sub(" \n ", "", sentences)
        sentences = re.sub(footer_pattern, ' ', sentences)
        citations = re.findall(citation_pattern, sentences) 
        rules = re.split(citation_pattern, sentences)
        print(citations)
        rule_index = 0
        for citation_index, citation in enumerate(citations):
            citation_type = "single"
            suffix = None
            if len(rules[citation_index]) > 3:
                rule = rules[citation_index]
                #citation_str.append({"rule_num": rule_num,"text":rules[citation_index][-1]})
                #suffix check
                if len(rules) > citation_index + 1:
                    #print(re.findall(suffix_pattern, rules[citation_index+1]))
                    #print(rules[citation_index+1])
                    #print(rules[citation_index+1].lstrip("[ \.,:;()]*").startswith(suffix_pattern_list))
                    pass
                rule_index = citation_index
            else:
                # multicitation or in sentence citation
                if len(rules[citation_index]) == 0 and citation_index == 0: # in sentence citation, just direct to the next sentence
                    rule_index += 1
                    rule = rules[rule_index]
                    citation_type = "in_sentence"
                else:
                    citation_type = "multi"
                    #result_dic[-1]['citation_type'] = citation_type
                    #if len(rules[citation_index+1]) >  0:
                    #    citation_str.append({"rule_num": rule_num,"text":rules[citation_index+1][:max(1, len(rules[citation_index+1]))]})
                    #if len(rules) - 1 > citation_index and re.findall("[where]|[which]|[in which].*",rules[citation_index+1]):
                    #    suffix = rules[citation_index+1]
                    rule = rules[rule_index] # multi citation, just keep the old sentence

In [ ]:
http_pattern = r'http[s]?:\/\/hudoc.echr.coe.int\/[a-z]+\?i\=\d+\-\d+'
website_pattern = r'([^ :]*):\/\/(?:([^:]*):([^@]*)@|)([^/:]{1,}):?(\d*)?(\/[^? ]*)\??((?:[^=&# ]*=?[^&# ]*&?)*)#?([^ ]*)?'
footer_pattern = r"Guide on Article \d+ of the Convention \- [A-Za-z ]+ European Court of Human Rights \d+\/\d+ Last update: \d+.\d+.\d+"

In [ ]:
sample_text = "The case of Ribcheva and Others v. Bulgaria concerned the death of a law-enforcement officer during a planned operation. The Court considered that,  \nCentre for Legal Resources on behalf of Valentin Campeanu v. Romania [GC], 2014, SSSS 131 and 143-144,\nMalik Babayev v. Azerbaijan [GC] (no. 12) )) )) ) or contractual military service (Boychenko v. Russia, 2021). Powell v. the United Kingdom (II) (dec.)) and that an issue may arise under Article 2 where it is shown that the authorities of a Contracting State have put an individual \n Marius Alexandru and Marinela Stefan v. Romania (dec.), SSSS 103-104 and  12-43 and SS"

In [ ]:
citation_pattern = "[A-Z]+[A-Za-z.\- ]*v\.[A-Za-z.\- ]+[GCIdecmeritsrevision\[\]().\d\- ]*[,S ]*[\-\d ]*[and]{0,3}[\-\d ]*"

In [ ]:
re.findall(citation_pattern, sample_text)

In [ ]:
first_article

In [ ]:
first_article_header = {}
p = re.compile('[A-Z]+\.  [a-zA-Z0-9 ]+')
for m in p.finditer(first_article):
    first_article_header = { 'article_header': m.group(), 'start': m.start()}
    break
#sections+=splitted_text

In [ ]:
first_article_header

In [ ]:
first_article_header_text = first_article[len(first_article_header['article_header']):]

In [ ]:
first_article_header_text = first_article_header_text.replace("\n", "")
first_article_header_text

In [ ]:
s1_citation = '\([A-Za-z. ]+, § \d+\)'
s2_citation = "\([A-Za-z. ]+ [[A-Z]*], § \d+\)"
s3_citation = "\([A-Za-z. ]+ \([A-Za-z]+\.\)\)"

In [ ]:
citations = re.findall(s1_citation + "|" + s2_citation, first_article_header_text)
citations

In [ ]:
texts = re.split(s1_citation + "|" + s2_citation, first_article_header_text)

In [ ]:
rules = [rule for rule in texts if len(rule) > 5]
rules

In [ ]:
data = []
for citation, rule in zip(citations, rules):
    data.append({"article_header": first_article_header['article_header'], "sentence": rule, "citation": citation})
data

In [ ]:
second_article

In [ ]:
p = re.compile('[A-Z]+\.  [a-zA-Z0-9 ]+')
for m in p.finditer(second_article):
    second_article_header = m.group()

In [ ]:
second_article_header

In [ ]:
second_article_text = second_article[len(second_article_header):]
second_article_text = second_article_text.replace("\n", "")
second_article_text

In [ ]:
citations = re.findall(s1_citation + "|" + s2_citation + "|" + s3_citation, second_article_text)
citations

In [ ]:
texts = re.split(s1_citation + "|" + s2_citation + "|" + s3_citation, second_article_text)

In [ ]:
rules = [rule for rule in texts if len(rule) > 5]
rules

In [ ]:
for citation, rule in zip(citations, rules):
    data.append({"article_header": second_article_header, "sentence": rule, "citation": citation})

In [ ]:
third_article

In [ ]:
third_article_subs = re.split("\n\n\d+\.  ", third_article)

In [ ]:
p = re.compile('[A-Z]+\.  [a-zA-Z0-9 ]+')
for m in p.finditer(third_article):
    third_article_header = m.group()

In [ ]:
third_article_header

In [ ]:
third_article_text = third_article[len(third_article_header):]

In [ ]:
len(third_article_subs)

In [ ]:
for sub in third_article_subs:
    if sub == third_article_header:
        third_article_subs.remove(sub)

In [ ]:
len(third_article_subs)

In [ ]:
for index, sub in enumerate(third_article_subs):
    third_article_subs[index] = sub.replace("\n", "")
third_article_subs

In [ ]:
pattern = r'http:\/\/hudoc.echr.coe.int\/[a-z]+\?i\=\d+\-\d+'
footer = r"Guide on Article \d+ of the Convention \- [A-Za-z ]+ European Court of Human Rights \d+\/\d+ Last update: \d+.\d+.\d+"

In [ ]:
for index, sub in enumerate(third_article_subs):
    #print(index)
    #print(sub)
    third_article_subs[index] = re.sub(pattern, '', third_article_subs[index])
    third_article_subs[index] = re.sub(footer, '', third_article_subs[index])
    third_article_subs[index] = unidecode.unidecode(third_article_subs[index])
    

In [ ]:
third_article_subs

In [ ]:
s1_citation = '[A-Za-z. ]+, SS \d+'
s2_citation = "[A-Za-z. ]+ [[A-Z]*], SS \d+"
s3_citation = "[A-Za-z. ]+ \([A-Za-z]+\.\)"
s4_citation = "[ ]*[A-Za-z. ]+[[A-Z]*]*, [S]+ [\d+]+[\-]*[\d+]*[;]*"
generic = s1_citation + "|" + s2_citation + "|" + s3_citation
multi_citation_pattern = "[A-Za-z. ]*[\[GC\]]*[\(dec.\)]*, S{2,4} \d+\-*\d*;?"

In [ ]:
case_name = "[A-Za-z. ]"
gc = "\[GC\]"
dec = "\(dec\)"
par_sign = "S"
par_num = "\d+\-*\d*;*"
citation_pattern = "[A-Za-z. ]*[\[GC\]]*[\(dec.\)]*,*[ \d]*, S{2,4} \d+\-*\d*;?"

In [ ]:
citations = [re.findall(citation_pattern, sub) for sub in third_article_subs]
citations

In [ ]:
rules = [re.split(citation_pattern, sub) for sub in third_article_subs]
rules

In [ ]:
result_dic = []
rule_num = 0
for corrected_rules, rule_citation in zip(rules, citations):
    corrected_rules = corrected_rules[:-1]
    for rule, citation in zip(corrected_rules, rule_citation):
        if len(rule) > 0:
            result_dic.append({"rule_num": rule_num, "sentence": rule, "citation": citation})
        else:
            result_dic.append({"rule_num": rule_num, "sentence": result_dic[-1]['sentence'], 'citation': citation})
    rule_num += 1

result_dic


In [ ]:
data = []
for result in result_dic:
    data.append({"article_header": third_article_header, "rule_num": result['rule_num'], "sentence": result['sentence'], "citation": result['citation']})

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.to_csv("citation_oriented.csv")

In [ ]:
data